In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
# df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\LTP_Combined.csv")
# FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_ltp_data")

In [4]:
# q = """
# WITH first_ltp_season AS
#     (SELECT
#          lower(parent_email) as parent_email, min(season) as season, min(transaction_date) as transaction_date
#     FROM
#          custom.ltp_combined_data
#     GROUP BY
#          parent_email),
# hist_acct_ids AS
#     (SELECT
#         acct_id, lower(email_addr) as email, date(add_date) as add_date
#     FROM
#         custom.arch_final_v_customer),
# hist_emails_and_accounts AS
#     (SELECT
#          parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
#          CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
#     FROM
#         first_ltp_season
#     LEFT JOIN
#         hist_acct_ids on first_ltp_season.parent_email = hist_acct_ids.email
#     WHERE
#         parent_email is not null and parent_email <> ''
#     GROUP BY
#         parent_email, acct_id, add_date),
# historical_data AS
#     (SELECT
#         hist_emails_and_accounts.acct_id, parent_email, hist_emails_and_accounts.season, transaction_date, hist_emails_and_accounts.add_date,
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2017-18' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2017_18",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2018-19' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2018_19",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2019-20' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2019_20",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2020-21' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2020_21",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2021-22' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2021_22",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2022-23' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2022_23",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2017-18' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2017_18",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2018-19' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2018_19",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2019-20' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2019_20",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2020-21' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2020_21",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2021-22' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2021_22",
#         CASE
#             WHEN cth_historical_all_1718_2223.season like '2022-23' then gross_revenue
#             ELSE 0
#         END AS "gross_rev_2022_23"
#     FROM
#         hist_emails_and_accounts
#     LEFT JOIN
#         custom.cth_historical_all_1718_2223 on hist_emails_and_accounts.acct_id = cth_historical_all_1718_2223.acct_id),
# historical_agg AS
#     (SELECT
#         parent_email, season, transaction_date, min(add_date) as add_date, max(is_stm_2017_18) as is_stm_2017_18,
#         max(is_stm_2018_19) as is_stm_2018_19,
#         max(is_stm_2019_20) as is_stm_2019_20, max(is_stm_2020_21) as is_stm_2020_21,
#         max(is_stm_2021_22) as is_stm_2021_22, max(is_stm_2022_23) as is_stm_2022_23,
#         sum(gross_rev_2017_18) as gross_rev_2017_18, sum(gross_rev_2018_19) as gross_rev_2018_19,
#         sum(gross_rev_2019_20) as gross_rev_2019_20, sum(gross_rev_2020_21) as gross_rev_2020_21,
#         sum(gross_rev_2021_22) as gross_rev_2021_22, sum(gross_rev_2022_23) as gross_rev_2022_23
#     FROM
#         historical_data
#     GROUP BY
#         parent_email, season, transaction_date),
# current_acct_ids AS
#     (SELECT
#          crm_id as acct_id, lower(email) as email, date(creation_datetime) as add_date
#     FROM
#         custom.seatgeek_v_clients),
# current_emails_and_accounts AS
#     (SELECT
#          parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
#          CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
#     FROM
#         first_ltp_season
#     LEFT JOIN
#         current_acct_ids on first_ltp_season.parent_email = current_acct_ids.email
#     WHERE
#         parent_email is not null and parent_email <> ''
#     GROUP BY
#         parent_email, acct_id, add_date),
# current_data AS
#     (SELECT
#          acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
#          current_emails_and_accounts.add_date, cth_v_ticket_2324.gross_revenue as gross_rev_2023_24,
#         CASE
#             WHEN cth_v_ticket_2324.ticket_type IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2023_24"
#     FROM
#         current_emails_and_accounts
#     LEFT JOIN
#         custom.cth_v_ticket_2324 on cth_v_ticket_2324.purch_client_crm_id = current_emails_and_accounts.acct_id),
# future_data AS
#     (SELECT
#          acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
#          current_emails_and_accounts.add_date, cth_v_ticket_subscription_2425.gross_revenue as gross_rev_2024_25,
#         CASE
#             WHEN cth_v_ticket_subscription_2425.ticket_type IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
#             ELSE 0
#         END AS "is_stm_2024_25"
#     FROM
#         current_emails_and_accounts
#     LEFT JOIN
#         custom.cth_v_ticket_subscription_2425 on cth_v_ticket_subscription_2425.purch_client_crm_id = current_emails_and_accounts.acct_id),
# current_agg AS
#     (SELECT
#         current_data.parent_email, current_data.season, current_data.transaction_date, min(current_data.add_date) as add_date,
#         max(is_stm_2023_24) as is_stm_2023_24, sum(gross_rev_2023_24) as gross_rev_2023_24
#     FROM
#         current_data
#     GROUP BY
#         current_data.parent_email, current_data.season, current_data.transaction_date),
# future_agg AS
#     (SELECT
#         parent_email, season, transaction_date, min(add_date) as add_date,
#         max(is_stm_2024_25) as is_stm_2024_25, sum(gross_rev_2024_25) as gross_rev_2024_25
#     FROM
#         future_data
#     GROUP BY
#         parent_email, season, transaction_date)
# SELECT
#     historical_agg.parent_email, historical_agg.season,
#     historical_agg.transaction_date, coalesce(historical_agg.add_date, current_agg.add_date) as add_date,
#     is_stm_2017_18, is_stm_2018_19,
#     is_stm_2019_20, is_stm_2020_21, is_stm_2021_22, is_stm_2022_23, is_stm_2023_24, is_stm_2024_25,
#     gross_rev_2017_18, gross_rev_2018_19, gross_rev_2019_20,
#     gross_rev_2020_21, gross_rev_2021_22, gross_rev_2022_23, gross_rev_2023_24, gross_rev_2024_25
# FROM
#     historical_agg
# LEFT JOIN
#     current_agg on historical_agg.parent_email = current_agg.parent_email
# LEFT JOIN
#     future_agg on historical_agg.parent_email = future_agg.parent_email
# ORDER BY
#     parent_email;
# """

In [5]:
q = """
WITH initial AS (
    SELECT
        LOWER(parent_email) AS parent_email,
        MIN(season) AS season,
        MIN(transaction_date) AS transaction_date
    FROM
        custom.community_ltp_data
    GROUP BY
        parent_email
    ),
acct_rev_info AS (
    SELECT
        contacts.email,
        createdon AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
    ),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
    )
SELECT
    parent_email,
    transaction_date,
    initial.season,
    add_date,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425
FROM
    initial
LEFT JOIN
    acct_rev_agg ON initial.parent_email = acct_rev_agg.email
WHERE
    initial.parent_email IS NOT NULL 
    AND initial.parent_email <> ''
    AND initial.season IN ('2021-22', '2022-23', '2023-24')
"""

In [6]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [7]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_17040\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [8]:
df['pre_altp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_altp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [9]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):8])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[8:(8+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(8 + value):12])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_tickets'] =  df[col_list].sum(axis = 1)

In [11]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_v_ltp")